# Marta Balairón García: 100451724
## INDEX: The loans programm
### 1) Linear programming approach
- Problem statment
- Problem Formulation
- Coding the problem
- Results interpretation
- Sensitivity analysis

### 2) Binary programming approach
####     2.1) Where to send students?
- Problem statment
- Problem Formulation
- Coding the problem
- Results interpretation

####     2.2) In which countries carry out the loan program?
- Problem statment
- Problem Formulation
- Coding the problem
- Results interpretation

# Canada, the student's dream.

In this modern world, learning **English** is a priority and the majority of the people agree on the idea that studying abroad is essential for having a valuable English level. Unfortunately, this is not possible for an important part of people.

Therefore, we are working on a program of loans to send students from different nationalities to Canada during two years.


<img src="img/flag.png" alt="Table2" style="width: 400px; height: 200px;" />

## PROBLEM STATEMENT

### What is the loan program about?


This loan's program is destinated to 6 different nationalities:
- Spanish (1) 
- English (2) 
- Polish (3) 
- French (4)
- Ucranian (5)
- German (6)

And we offer 6 different kind of loans depending on the economic capabilities of each family:
- Type 1: 5.000€
- Type 2: 10.000€
- Type 3: 15.000€
- Type 4: 20.000€
- Type 5: 25.000€
- Type 6: 30.000€

### How is the money distributed?

We count with a **budget of 18.000.000€**. This money is reparted in order of **maximize** the number of money distributed in  loans but following some conditions:
- Ucranian people must receive at least 5 type 6 loans (due to the difficult conditions of life there)
- Polish people must receive at least 5 type 4, 5 and 6 loans (due to the different of economic power with the other countries)
- At least, the number of distributed loans of each type given to a country must be equal or greater than the number of requests made for that country for that type of loan.

It is important also to consider the requests of the people which are represented in the following table.


<img src="img/requests.png" alt="Table2" style="width: 600px; height: 200px;" />

As well each country has a limit of budget for loans which cannot exceed depending in their economic situation:
- Spain: 2.000.000 €
- England: 1.500.000 €
- Poland: 5.000.000 €
- France: 1.500.000 €
- Ucrain: 7.000.000 €
- Germany: 1.000.000 €

When the loans are distributed, if there are more loans distributed than the ones requested, people can ask again for them in case they follow the conditions.

## PROBLEM FORMULATION

### Sets

 $N$ = set of nationalities
 
 $L$= set of loan type's
 

### Variables

 $x_{ij}$ = number of loans given for someone of nationality $i$ of type $j$
 
### Parameters

$quantity$ = money paid for each loan type $j$, $\forall j\in L$

$request$ = number of loans of type $j$ requested by nationality $i$, $\forall j\in L$ $\forall i\in N$

$limit$ = maximum amount of money that a country $i$ can receive for loans,  $\forall i\in N$


### Objective

Maximize the number of budget distributed:
 $\max \sum_{i\in N}\sum_{j \in L} x_{ij}*quantity_{j}$


### Constraints

The money distributed can not exceed the total budget:
* $\sum_{i\in N}\sum_{j \in L} quantity_{j}* x_{ij} \leq 18.000.000 $, $\forall i \in N$, $\forall j \in L$

The loans of each type given to each country, must be at least equal to the number of requested loans by students.
* $x_{ij} \geq request_{ij}$ $\forall i \in N$, $\forall j \in L$

The amount of money given in loans for each contry cannot exceed the maximum amount money stablished:
* $\sum_{j \in L} x_{ij}* quantity_{j} \leq limit_{j} $, $\forall i \in N$

Ucranian people must receive at least five type 6 loans:
* $x_{56} \geq 5 $
 
Polish people must receive at least five type 4, 5 and 5 loans:
* $x_{34} + x_{35} + x_{36} \geq 5 $



  ## CODING THE PROBLEM

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [2]:
# We import the necessary dataframes created in excel for initializing the variables
import pandas as pd

df_requests = pd.read_excel("requests.xlsx")
df_quantities = pd.read_excel("quantity.xlsx")
df_limits = pd.read_excel("limits.xlsx")

In [3]:
# we create the necessary functions to initiallize the variables
def requ(model):
    requests = dict()
    for i in model.i:
        for j in model.j:
            j_s = str(j)
            requests[(i, j)] = df_requests[j_s][i-1]           
    return requests

def quant(model):
    quantities = dict()
    for j in model.j:
        quantities[j] = df_quantities["quantity"][j-1]
    return quantities

def limits(model):
    limits = dict()
    for i in model.i:
        limits[i] = df_limits["limit"][i-1]
    return limits



In [4]:
# Define the data
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT) # important for sensitivity

model.i = Set(initialize=[1, 2, 3, 4, 5, 6], doc='nationalities' ) 
model.j = Set(initialize=[1, 2, 3, 4, 5, 6], doc='loan_types') 

quantities = quant(model)
limits = limits(model)
requests = requ(model)

model.quantity = Param(model.j, initialize=quantities, doc="quantity assigned for each loan type")
model.limits = Param(model.i, initialize=limits, doc="limit for amount of money distributed for each country")
model.requests = Param(model.i, model.j, initialize=requests, doc='number loans requested') 

#### Declare the variables:

In [5]:
model.x = Var(model.i, model.j, doc="number of type j loans given to nationality i", within=NonNegativeReals)

#### Define objective function:

In [6]:
# function for defaining the objective function
def objective_rule(model):
    suma = 0
    for i in model.i:
        for j in model.j:
            suma += model.x[i, j]*model.quantity[j]
    return suma
model.objective = Objective(rule=objective_rule, sense=maximize, doc='distributed loans')


In [7]:
#model.objective.pprint()

#### Define the constraints:

In [8]:
# function for defining the budget constraint
def budget(model):
    suma = 0
    for i in model.i:
        for j in model.j:
            suma += (model.x[i, j] * model.quantity[j])
            
    return suma


# function for defining the limit of loans contraints
def limit(model, i):
    suma = 0
    for j in model.j:
        suma += model.x[i, j]*model.quantity[j]  
    
    return suma



In [9]:
# budget constraint
eq = budget(model)
model.budget = Constraint(expr = eq <= 18000000)

# limit of amount of money distributed:
eq1 = limit(model, 1)
eq2 = limit(model, 2)
eq3 = limit(model, 3)
eq4 = limit(model, 4)
eq5 = limit(model, 5)
eq6 = limit(model, 6)

model.limit_constraint = ConstraintList()
model.limit_constraint.add(expr = eq1 <= model.limits[1])
model.limit_constraint.add(expr = eq2 <= model.limits[2])
model.limit_constraint.add(expr = eq3 <= model.limits[3])
model.limit_constraint.add(expr = eq4 <= model.limits[4])
model.limit_constraint.add(expr = eq5 <= model.limits[5])
model.limit_constraint.add(expr = eq6 <= model.limits[6])

# Reuests constraints
model.requests_Spain_constraint = ConstraintList()
model.requests_Spain_constraint.add(expr = (model.x[1, 1] >= model.requests[1, 1]))
model.requests_Spain_constraint.add(expr = (model.x[1, 2] >= model.requests[1, 2]))
model.requests_Spain_constraint.add(expr = (model.x[1, 3] >= model.requests[1, 3]))
model.requests_Spain_constraint.add(expr = (model.x[1, 4] >= model.requests[1, 4]))
model.requests_Spain_constraint.add(expr = (model.x[1, 5] >= model.requests[1, 5]))
model.requests_Spain_constraint.add(expr = (model.x[1, 6] >= model.requests[1, 6]))

model.requests_England_constraint = ConstraintList()
model.requests_England_constraint.add(expr = (model.x[2, 1] >= model.requests[2, 1]))
model.requests_England_constraint.add(expr = (model.x[2, 2] >= model.requests[2, 2]))
model.requests_England_constraint.add(expr = (model.x[2, 3] >= model.requests[2, 3]))
model.requests_England_constraint.add(expr = (model.x[2, 4] >= model.requests[2, 4]))
model.requests_England_constraint.add(expr = (model.x[2, 5] >= model.requests[2, 5]))
model.requests_England_constraint.add(expr = (model.x[2, 6] >= model.requests[2, 6]))

model.requests_Poland_constraint = ConstraintList()
model.requests_Poland_constraint.add(expr = (model.x[3, 1] >= model.requests[3, 1]))
model.requests_Poland_constraint.add(expr = (model.x[3, 2] >= model.requests[3, 2]))
model.requests_Poland_constraint.add(expr = (model.x[3, 3] >= model.requests[3, 3]))
model.requests_Poland_constraint.add(expr = (model.x[3, 4] >= model.requests[3, 4]))
model.requests_Poland_constraint.add(expr = (model.x[3, 5] >= model.requests[3, 5]))
model.requests_Poland_constraint.add(expr = (model.x[3, 6] >= model.requests[3, 6]))

model.requests_France_constraint = ConstraintList()
model.requests_France_constraint.add(expr = (model.x[4, 1] >= model.requests[4, 1]))
model.requests_France_constraint.add(expr = (model.x[4, 2] >= model.requests[4, 2]))
model.requests_France_constraint.add(expr = (model.x[4, 3] >= model.requests[4, 3]))
model.requests_France_constraint.add(expr = (model.x[4, 4] >= model.requests[4, 4]))
model.requests_France_constraint.add(expr = (model.x[4, 5] >= model.requests[4, 5]))
model.requests_France_constraint.add(expr = (model.x[4, 6] >= model.requests[4, 6]))

model.requests_Ucrain_constraint = ConstraintList()
model.requests_Ucrain_constraint.add(expr = (model.x[5, 1] >= model.requests[5, 1]))
model.requests_Ucrain_constraint.add(expr = (model.x[5, 2] >= model.requests[5, 2]))
model.requests_Ucrain_constraint.add(expr = (model.x[5, 3] >= model.requests[5, 3]))
model.requests_Ucrain_constraint.add(expr = (model.x[5, 4] >= model.requests[5, 4]))
model.requests_Ucrain_constraint.add(expr = (model.x[5, 5] >= model.requests[5, 5]))
model.requests_Ucrain_constraint.add(expr = (model.x[5, 6] >= model.requests[5, 6]))

model.requests_Germany_constraint = ConstraintList()
model.requests_Germany_constraint.add(expr = (model.x[6, 1] >= model.requests[6, 1]))
model.requests_Germany_constraint.add(expr = (model.x[6, 2] >= model.requests[6, 2]))
model.requests_Germany_constraint.add(expr = (model.x[6, 3] >= model.requests[6, 3]))
model.requests_Germany_constraint.add(expr = (model.x[6, 4] >= model.requests[6, 4]))
model.requests_Germany_constraint.add(expr = (model.x[6, 5] >= model.requests[6, 5]))
model.requests_Germany_constraint.add(expr = (model.x[6, 6] >= model.requests[6, 6]))

# Ucrain constraint
model.ucrain = Constraint(expr = model.x[5, 6] >= 5)

# Poland constraint
model.poland = Constraint(expr = (model.x[3, 4] + model.x[3, 5] + model.x[3, 6]) >= 5)



#### Solving the problem:

In [10]:
Solver = SolverFactory("glpk")
results = Solver.solve(model)
#model.display()

## RESULT INTERPRETATION

<img src="img/results.png" alt="Table2" style="width: 650px; height: 400px;" />

In this graph we can observe how are the loans distributed. It is easy to see that:
- Most of the loans are of **type 1**, since they have the least restrictive requirements and therefore are the most requested ones.
- In general, the countries which more loans received are **Poland** and **Ucrain** since they are the one which have a more depressed economy and therfore, their limit budget is higher.

In the following graphs we can analyse how specifically each type of loan is distributed:

<img src="img/t12.png" alt="Table2" style="width: 750px; height: 200px;" /> 


<img src="img/t 3 4.png" alt="Table2" style="width: 750px; height: 200px;" />

<img src="img/t 5 6.png" alt="Table2" style="width: 750px; height: 200px;" />


Moreover we have calculated that the total budget used is **18.000.000 €**, and as a feasible solution was found, means that **all requests were satisfyied**. This results are great since we have maximize the budget as much as possible and since we have satisfyied all requests following the constraints.

## SENSITIVITY ANALYSIS

In [11]:
model.dual.display()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                            : Value
                            budget :   1.0
               limit_constraint[1] :  -0.0
               limit_constraint[2] :  -0.0
               limit_constraint[3] :  -0.0
               limit_constraint[4] :  -0.0
               limit_constraint[5] :  -0.0
               limit_constraint[6] :   0.0
                            poland :   0.0
    requests_England_constraint[1] :  -0.0
    requests_England_constraint[2] :  -0.0
    requests_England_constraint[3] :  -0.0
    requests_England_constraint[4] :   0.0
    requests_England_constraint[5] :  -0.0
    requests_England_constraint[6] :  -0.0
     requests_France_constraint[1] :   0.0
     requests_France_constraint[2] :  -0.0
     requests_France_constraint[3] :  -0.0
     requests_France_constraint[4] :  -0.0
     requests_France_constraint[5] :  -0.0
     requests_France_constraint[6] :  -0.0
    requests_Germany_constraint[1] :   0.0


**The sensitivity** shows the influence of each of the constraints in the objective. Every time you increment one unit of a variable, the number that appears shows how much increases or decreases the distributed money. 

## BINARY PROBLEM STATMENT

Once the loans are distributed in the most efficient way, it is time to decide how each nationality student is allocated in the different cities of Canada. For this international program are 6 different cities where you can go:
- Quebec (1)
- Toronto (2) 
- Vancuver (3) 
- Montreal (4)
- Ontario (5)
- New Brunswick (6)
- Ottawa (7)
- Victoria (8)

What is more, Quebec, Ontario and New Brunswick are consider Francophone cities of the countries.

The **objective** of the program is to encourage the internationalization of the students so we are looking to **maximize the number of cities for each nationality**. However, there exist some limitations to this:
- English students (2), because of reasons of language, cannot go to not francophone cities. (2, 3, 4, 7, 8)
- French students (4), simmilarly cannot go to francophone cities. (1, 5, 6)
- All cities must receive at least 2 different nationalities.
- Because governamental reasons, there cannot be students of the same nationality in Vancuver(3) and Toronto(2).
- Victoria city (8) it is not capable of host more than 3 nationalities because it doesnt have enough hall of students.


## BINARY PROBLEM FORMULATION (I)

### Sets

 $O$ = set of origin nationalities
 
 $D$= set of destiny cities.
 

### Variables

$$
x_{ij}=
\left\{\begin{array}{ll} 
1, & \text{if the city $j$ receive students from nationality $i$,}\\
0, & \text{if the city $j$ does not receive students from nationality $i$}
\end{array} \right.\quad
$$

 

### Objective

Maximize the distribution of different nationalities in different cities:
* $\max \sum_{i\in O}\sum_{j \in L} x_{ij}$


### Constraints

English students cannot attend to not francophone cities:
* $x_{22} + x_{23} + x_{24} + x_{27} + x_{28} = 0$ 


French students cannot attend to  francophone cities:
* $x_{41} + x_{45} + x_{46} = 0$ 

Each city must receive at least 2 nationalities:
* $\sum_{i \in O} x_{ij} \geq 2 $, $\forall j \in D$

Incompatibility of same nationality at Vancuver and Toronto: 
* $x_{i3} + x_{i2} \leq 1$ $\forall i \in O$
 
Limitation at hosting of Victoria city:
* $\sum_{i \in O} x_{i8} \leq 3 $


## CODING THE BINARY PROBLEM (I)

In [12]:
model_2 = ConcreteModel("Binary loans model")

model_2.i = RangeSet(1, 6) # origin
model_2.j = RangeSet(1, 8) # destiny

#### Define the variables

In [13]:
model_2.x = Var(model_2.i, model_2.j, domain=Binary)

#### Objective function

In [14]:
def objective(model):
    suma = 0
    for i in model.i:
        for j in model.j:
            suma += model.x[i, j]
    return suma

eq = objective(model_2)
model_2.obj = Objective(rule=eq, sense=maximize)

In [15]:
# model_2.obj.pprint()

#### Define the constraints

In [16]:
# functions for defining the equations of the constraints
def minimum(model, j):
    suma = 0
    for i in model.i:
        suma += model.x[i, j]  
    return suma

def victoria(model):
    suma = 0
    for i in model.i:
        suma += model.x[i, 8]
    
    return suma

In [17]:
# English and French constraints
model_2.eng = Constraint(expr=(model_2.x[2, 2] + model_2.x[2, 3] + model_2.x[2, 4] + model_2.x[2, 7] + model_2.x[2, 8]) == 0)
model_2.fr = Constraint(expr=(model_2.x[4, 1] + model_2.x[4, 5] + model_2.x[4, 6]) == 0)

# minimum adjudications to the destiny constraints
eq1_min = minimum(model_2, 1)
eq2_min = minimum(model_2, 2)
eq3_min = minimum(model_2, 3)
eq4_min = minimum(model_2, 4)
eq5_min = minimum(model_2, 5)
eq6_min = minimum(model_2, 6)
eq7_min = minimum(model_2, 7)
eq8_min = minimum(model_2, 8)

model_2.min1 = Constraint(expr=(eq1_min >= 2))
model_2.min2 = Constraint(expr=(eq2_min >= 2))
model_2.min3 = Constraint(expr=(eq3_min >= 2))
model_2.min4 = Constraint(expr=(eq4_min >= 2))
model_2.min5 = Constraint(expr=(eq5_min >= 2))
model_2.min6 = Constraint(expr=(eq6_min >= 2))
model_2.min7 = Constraint(expr=(eq7_min >= 2))
model_2.min8 = Constraint(expr=(eq8_min >= 2))

# incopatibilities
model_2.incop1 = Constraint(expr=((model_2.x[1, 3] + model_2.x[1, 2]) <= 1))
model_2.incop2 = Constraint(expr=((model_2.x[2, 3] + model_2.x[2, 2]) <= 1))
model_2.incop3 = Constraint(expr=((model_2.x[3, 3] + model_2.x[3, 2]) <= 1))
model_2.incop4 = Constraint(expr=((model_2.x[4, 3] + model_2.x[4, 2]) <= 1))
model_2.incop5 = Constraint(expr=((model_2.x[5, 3] + model_2.x[5, 2]) <= 1))
model_2.incop6 = Constraint(expr=((model_2.x[6, 3] + model_2.x[6, 2]) <= 1))

# minimum Victoria city
eq_vict = victoria(model_2)
model_2.vict = Constraint(expr=(eq_vict <= 3))


#### Model resolution:

In [18]:
Solver = SolverFactory("glpk")
results = Solver.solve(model_2)
#model_2.display()

## Analysis of the results (I)

After having optimized the problem we can conclude that the students from the different nationalities will be distributed this way:
- Spanish students will be sent to Quebec, Toronto and Ottawa.
- English Students will be sent to Ontario and New Brunswick.
- Polish students will be sent to Quebec, Toronto, Montreal, Ontario, New Brunswick, Ottawa and Victoria.
- French students will be sent to Vancuver, Montreal, Ottawa and Victoria city.
- Ucranian students will be sent to Quebec, Vancuver, Montreal, Ontario, New Brunswick, Ottawa and Victoria
- Germany Students will be sent to Toronto and Ottawa.

## AMPLIATION OF THE BINARY STATMENT

Sudenly, an economical problem has occur and now the loans comapy cannot invest in all the countries. Therefore, now we must select in which countries take the loans project. We must that we want to invest in the maximum number of projects as possible. 

We are going to distributed to the maximum number of countries, the complete assignation of loans calculated in the first part of this project. (For this, we are going to load the data which is stored in an excel df).

We want to distribute the **maximum number of complete loan programs** as possible (implicetily, we want to use as much avaliable money as possible) , but taking into account that **the new budget is 12.000.000** and the following constraints:
- The distributed money, cannot exceed the new budget
- Since Germany and England have the most powerful economic resources, we cannot invest in both projects.
- If we invest in Germany or England, we must also invest in Ucrain and/or Poland (since they ave the most depressed economies).
- Spain and France ar consider simmilar countries regarding the economy, therefore we must invest in at least one of those.

## BINARY PROBLEM FORMULATION (II)


### Sets

 $N$ = set of candidate nationalities
 $L$ = set of types of loans

### Parameters

$quantity_{j}$ = money paid for each loan type $j$, $\forall L$

$assignation_{ij}$ = number of loans of type $j$ assigned to nationality $i$, $\forall L$

$expenses_{i}$ = expenses generated by loans given to nationality $i$, $\forall N$

$expenses_{i}$ = $\sum_{j\in L}$ $assignation_{ij}$ * $quantity_{j}$ $\forall i \in N$

### Variables

$$
x_{i}=
\left\{\begin{array}{ll} 
1, & \text{invest at project $i$,}\\
0, & \text{don not invest at project $i$}
\end{array} \right.\quad
$$

### Objective

Maximize the number of inversions (implicetely, maximize the used budget):
* $\max \sum_{i\in N}x_{i}$


### Constraints


Budget constraint:
*  $\sum_{i\in N} expenses_{i}* x_{i} \leq 12.000.000 $, $\forall i \in N$

Incompatibility of Germany and Engalnd:
* $x_{6} + x_{2} \leq 1$ 


Dependency of Ucrain and Poland on England and Germany:
* $x_{6} + x_{3} + x_{5} \geq 2$ 
* $x_{4} + x_{3} + x_{5} \geq 2$ 

Obligation of apareance of France and Spain:
* $2 \geq x_{1} + x_{4} \geq 1$ 


## CODING THE BINARY PROBLEM (II)

In [19]:
df_assignations = pd.read_excel("assignment.xlsx")

model_3 = ConcreteModel("Binary loans plans selection")
model_3.i = RangeSet(1, 6) # nationalities
model_3.j = RangeSet(1, 6) # loan types

#### Define the variables:

In [20]:
# we create the necessary functions to initiallize the variables
def expenses(model): 
    assignations = dict()
    for i in model.i:
        for j in model.j:
            j_s = str(j)
            assignations[(i, j)] = df_assignations[j_s][i-1]       

    expenses = dict()
    for i in model.i:
        exp = 0
        for j in model.j:
            exp += assignations[(i, j)]*quantities[j]

        expenses[i] = exp
    
    return expenses  

In [21]:
expenses = expenses(model_3)

model_3.expenses = Param(model_3.i, initialize=expenses, doc="expenses done in the assignment of loans to that nationality")
model_3.x = Var(model_3.i, domain=Binary)

#### Define the objective

In [22]:
def obj(model):
    suma = 0
    for i in model.i:
        suma += model.x[i]
    return suma

eq = obj(model_3)
model_3.obj = Objective(rule=eq, sense=maximize)

In [23]:
#model_3.obj.pprint()

#### Define the constraints

In [24]:
# function for defining the budget constraint
def budget(model):
    suma = 0
    for i in model.i:
        suma += model.x[i]*model.expenses[i]
            
    return suma


In [25]:
# budget constraint
eq = budget(model_3)
model_3.budget_constraint = Constraint(expr=(eq <= 12000000))

# incompatibility Germany and England
model_3.incomp = Constraint(expr=(model_3.x[6] + model_3.x[2]) <= 1)

# dependency Germany and England with Ucrain and poland
model_3.dep1 = Constraint(expr=(model_3.x[6] + model_3.x[3] + model_3.x[5]) >= 2)
model_3.dep2 = Constraint(expr=(model_3.x[4] + model_3.x[3] + model_3.x[5]) >= 2)

# obligation of appareance of france and Spain

model_3.fs = Constraint(expr=(model_3.x[1] + model_3.x[4]) >= 1)


#### Solve the problem

In [26]:
Solver = SolverFactory("glpk")
results = Solver.solve(model_3)
model_3.display()

Model Binary loans plans selection

  Variables:
    x : Size=6, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   4.0

  Constraints:
    budget_constraint : Size=1
        Key  : Lower : Body      : Upper
        None :  None : 9500000.0 : 12000000.0
    incomp : Size=1
        Key  : Lower : Body : Upper
        None :  None :  1.0 :   1.0
    dep1 : Size=1
        Key  : Lower : Body : Upper
        None :   2.0 :  2.0 :  None
    dep2 : Size=1
        Key  : Lower : Body : Upper
        None 

## Analysis of the result (II)

After having optimize the model, we conclude that the most optimal is to carry out the loan program at **Spain, Poland, France and Germany**. Making use of 9.5 million € which is an **80% of the proposed budget**.

We can observe how all the constraints are accomplished:
- Germany receives de loans but not England.
- As Germany receives, Poland does it too.
- Both France and Spain receive loans.

